In [12]:

import sys
import os
sys.path.append(os.path.abspath("../"))

In [13]:
from BEACH.lib import *

In [26]:
c = np.array([[0,2.5],[2,5+10]])

In [27]:
import pandas as pd
df = pd.DataFrame(c, columns=['j', 'coef'])
df.to_csv('coef_file.csv', index=False)

In [29]:
df = pd.read_csv('coef_file.csv')
df

,j,coef
0,0.0,2.5
1,2.0,15.0
